In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle
try:
    from fastprogress.fastprogress import progress_bar as tqdm
except:
    from tqdm import tqdm

confirmed_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_cases = pd.read_csv(confirmed_cases_url, sep=',')
deaths_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths = pd.read_csv(deaths_url, sep=',')
path_to_save = '../../figures/'
path_data = '../../data/'

In [2]:
def delay_cases(new_I_t, len_new_I_t, len_new_cases_obs , delay, delay_arr):
    """
    Delays the input new_I_t by delay and return and array with length len_new_cases_obs
    The initial delay of the output is set by delay_arr. 
    
    Take care that delay is smaller or equal than delay_arr, otherwise zeros are 
    returned, which could potentially lead to errors

    Also assure that len_new_I_t is larger then len(new_cases_obs)-delay, otherwise it 
    means that the simulated data is not long enough to be fitted to the data.
    """
    delay_mat = make_delay_matrix(n_rows=len_new_I_t, 
                                  n_columns=len_new_cases_obs, initial_delay=delay_arr)
    inferred_cases = interpolate(new_I_t, delay, delay_mat)
    return inferred_cases 

def make_delay_matrix(n_rows, n_columns, initial_delay=0):
    """
    Has in each entry the delay between the input with size n_rows and the output
    with size n_columns
    """
    size = max(n_rows, n_columns)
    mat = np.zeros((size, size))
    for i in range(size):
        diagonal = np.ones(size-i)*(initial_delay + i)
        mat += np.diag(diagonal, i)
    for i in range(1, size):
        diagonal = np.ones(size-i)*(initial_delay - i)
        mat += np.diag(diagonal, -i)
    return mat[:n_rows, :n_columns]

def interpolate(array, delay, delay_matrix):
    interp_matrix = tt.maximum(1-tt.abs_(delay_matrix - delay), 0)
    interpolation = tt.dot(array,interp_matrix)
    return interpolation

In [3]:
import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

date_data_begin = datetime.date(2020,3,1)
date_data_end = datetime.date(2020,3,15)
num_days_to_predict = 28


diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
format_date = lambda date_py: '{}/{}/{}'.format(date_py.month, date_py.day,
                                                 str(date_py.year)[2:4])
date_formatted_begin = format_date(date_data_begin)
date_formatted_end = format_date(date_data_end)

cases_obs =  np.array(
    confirmed_cases.loc[confirmed_cases["Country/Region"] == "Germany", 
                        date_formatted_begin:date_formatted_end])[0]
#cases_obs = np.concatenate([np.nan*np.ones(diff_data_sim), cases_obs])
print('Cases yesterday ({}): {} and '
      'day before yesterday: {}'.format(date_data_end.isoformat(), *cases_obs[:-3:-1]))
num_days = (date_data_end - date_begin_sim).days
date_today = date_data_end + datetime.timedelta(days=1)
# ------------------------------------------------------------------------------ #
# model setup and training
# ------------------------------------------------------------------------------ #
np.random.seed(0)

def SIR_model(λ, μ, S_begin, I_begin, N):
    new_I_0 = tt.zeros_like(I_begin)
    def next_day(λ, S_t, I_t, _):
        new_I_t = λ/N*I_t*S_t
        S_t = S_t - new_I_t
        I_t = I_t + new_I_t - μ * I_t
        return S_t, I_t, new_I_t
    outputs , _  = theano.scan(fn=next_day, sequences=[λ], 
                               outputs_info=[S_begin, I_begin, new_I_0])
    S_all, I_all, new_I_all = outputs
    return S_all, I_all, new_I_all

with pm.Model() as model:
    # true cases at begin of loaded data but we do not know the real number
    I_begin = pm.HalfCauchy('I_begin', beta=100)

    # fraction of people that are newly infected each day
    λ = pm.Lognormal("λ", mu=np.log(0.4), sigma=0.5)

    # fraction of people that recover each day, recovery rate mu
    μ = pm.Lognormal('μ', mu=np.log(1/8), sigma=0.2)

    # delay in days between contracting the disease and being recorded
    delay = pm.Lognormal("delay", mu=np.log(8), sigma=0.2)

    # prior of the error of observed cases
    σ_obs = pm.HalfCauchy("σ_obs", beta=10)

    N_germany = 83e6

    # -------------------------------------------------------------------------- #
    # training the model with loaded data
    # -------------------------------------------------------------------------- #

    S_begin = N_germany - I_begin
    S_past, I_past, new_I_past = SIR_model(λ=λ * tt.ones(num_days-1), μ=μ, 
                                               S_begin=S_begin, I_begin=I_begin,
                                               N=N_germany)
    new_cases_obs = np.diff(cases_obs)
    new_cases_inferred = delay_cases(new_I_past, len_new_I_t=num_days - 1, 
                                     len_new_cases_obs=len(new_cases_obs), 
                                     delay=delay, delay_arr=diff_data_sim)

    # Approximates Poisson
    # calculate the likelihood of the model:
    # observed cases are distributed following studentT around the model
    pm.StudentT(
        "obs",
        nu=4,
        mu=new_cases_inferred,
        sigma=(new_cases_inferred)**0.5 * σ_obs,
        observed=new_cases_obs)  
    
    S_past = pm.Deterministic('S_past', S_past)
    I_past = pm.Deterministic('I_past', I_past)
    new_I_past = pm.Deterministic('new_I_past', new_I_past)
    new_cases_past = pm.Deterministic('new_cases_past', new_cases_inferred)
    

    # -------------------------------------------------------------------------- #
    # run model, pm trains and predicts when calling this
    # -------------------------------------------------------------------------- #
    
    time_beg = time.time()
    trace = pm.sample(draws=3000, tune=800, chains=2)
    print("Model run in {:.2f} s".format(time.time() - time_beg))

Cases yesterday (2020-03-15): 5795 and day before yesterday: 4585


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [σ_obs, delay, μ, λ, I_begin]
Sampling 2 chains, 8 divergences: 100%|██████████| 7600/7600 [01:30<00:00, 83.68draws/s] 
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.


Model run in 100.91 s


In [4]:
# brute force search, create log like matrix

load_from_file = False


import pickle

from itertools import product
from tqdm.notebook import tqdm
fn = model.fn(model.logpt)

print(model.vars)
points_list = [{"I_begin_log__" : 3, "λ_log__": -0.9, "μ_log__": -2.3, "delay_log__": 2, 
                "σ_obs_log__": 2}]

I_begin_r = np.linspace(np.log(5), np.log(100), 30)
lambda_r = np.linspace(np.log(0.1), np.log(0.6), 40)
mu_r = np.linspace(np.log(1 / 15), np.log(1 / 4), 30)
delay_r = np.linspace(np.log(5), np.log(15), 30)
sigma_obs_r = np.linspace(np.log(1), np.log(50), 20)


if not load_from_file:
    logp = []
    for values in tqdm(product(I_begin_r, lambda_r, mu_r, delay_r, sigma_obs_r), total = 30*40*30*30*20):
        point = {"I_begin_log__" : values[0], "λ_log__": values[1], "μ_log__": values[2], 
                            "delay_log__": values[3], "σ_obs_log__": values[4]}
        logp.append(fn(point))
    with open('logp.pickle', 'wb') as file:
        pickle.dump(logp, file)
else:
    logp = pickle.load(open("logp.pickle", "rb"))
        
logp_mat = np.array(logp).reshape((30,40,30,30,20))
        

par_titles = ["  I_0", "    λ", "    μ", "delay", "σ_obs"]

par_values = []
par_values.append( np.exp(I_begin_r) )
par_values.append( np.exp(lambda_r) )
par_values.append( np.exp(mu_r) )
par_values.append( np.exp(delay_r) )
par_values.append( np.exp(sigma_obs_r) )

[I_begin_log__, λ_log__, μ_log__, delay_log__, σ_obs_log__]


In [5]:
# get percentiles from bayesian model run

# print(trace.varnames)
percentiles = np.ones((5,2))*np.nan
for idx, key in enumerate(["I_begin", "λ", "μ", "delay", "σ_obs"]):
    percentiles[idx, 0] = np.percentile(trace[key], q=2.5, axis=0)
    percentiles[idx, 1] = np.percentile(trace[key], q=97.5, axis=0)

print("percentiles:", percentiles)

logp_reference = np.max(trace.model_logp)
print(f"reference logp: {logp_reference:.4f}")


# get parameter combinations that are not within 95% ci of bayesian estimate but produce a decent likeliehood
special = []
for i in tqdm(range(0, len(logp))):
    idx = np.unravel_index(i, (30, 40, 30, 30, 20))
    this_logp = logp_mat[idx]
    
    if this_logp < logp_reference-3:
        continue
    
    for par in range(0,5):
        if (par_values[par][idx[par]] <= percentiles[par, 0] or
            par_values[par][idx[par]] >= percentiles[par, 1]):
            special.append(idx)
            
print(f"{len(special)} parameters resulted in log likeliehood above {logp_reference-3:.3f}")
    

percentiles: [[ 3.02864076 76.88057754]
 [ 0.31549841  0.51911545]
 [ 0.08263706  0.1791954 ]
 [ 5.81989883 12.73725286]
 [ 4.23401301 13.46223251]]
reference logp: -91.2765



4237 parameters resulted in log likeliehood above -94.276


In [6]:
# print the special guys

def is_within_ci(par, pos):
    if (par_values[par][pos] <= percentiles[par, 0] or
        par_values[par][pos] >= percentiles[par, 1]):
        return False
    return True

def print_par_from_pos(idx):
    for par in range(0,5):
        pos = idx[par]
        print(f" {'! ' if not is_within_ci(par, pos) else '  '} {par_titles[par]}: {par_values[par][pos]:.3f}")
    print("----------------")
    
special_par = []
super_special = []
print_all = False

for this_special in special:
    num = 0
    if print_all:
        print_par_from_pos(this_special)
        
    for par in range(0,5):
        pos = this_special[par]
        if not is_within_ci(par, pos):
            num += 1
            special_par.append(par)
            if num > 1:
                super_special.append(this_special)

                
# how many outliers belong to which parameter
y = np.bincount(special_par)
for par in range(0,5):
    print(f"{par_titles[par]} {y[par]}  (CI: [{percentiles[par, 0]:.2f}, {percentiles[pad, 1]:.2f}] Range: [{par_values[par][0]:.2f}, {par_values[par][-1]:.2f}])")
print("\n\n")

# when more than two parameters were out, print them
for this_special in super_special:
    print_par_from_pos(this_special)

  I_0 133
    λ 392
    μ 564
delay 959
σ_obs 2257



      I_0: 7.558
        λ: 0.415
        μ: 0.126
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 7.558
        λ: 0.415
        μ: 0.126
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 8.381
        λ: 0.415
        μ: 0.132
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 8.381
        λ: 0.415
        μ: 0.132
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 9.293
        λ: 0.397
        μ: 0.115
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 9.293
        λ: 0.397
        μ: 0.115
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 10.304
        λ: 0.397
        μ: 0.121
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 10.304
        λ: 0.397
        μ: 0.121
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 11.425
        λ: 0.397
        μ: 0.126
 !  delay: 5.818
 !  σ_obs: 4.226
----------------
      I_0: 11.425
       